# n-gram与顺序依赖

## (1)自然语言不是集合，而是序列：n-gram 是人类第一次试图“用有限窗口记住顺序”的工程妥协方案。

## (2)为什么「顺序」在 NLP 中是一等公民
**1.例子**：
- I do not like this movie
- I do like this movie not
这两句话词合集完全相同，语义却相反。

说明：
>“词是否出现” ≠ “语言含义”  “出现顺序”直接决定语义

**2.Bag-of-Words 的本质缺陷**
BoW将文本表示为各个单词出现次数的向量，它完全忽略词的顺序和上下文，仅统计词频：
$$sentence→multiset(words)$$

>BOW实际上是把一句话投影到一个“无序词袋空间”，导致彻底失去了序列结构

>bow本质假设了什么？
>假设词之间条件独立，与位置无关。

**3.n-gram**
1）语言模型想算的东西

给你一句话（一个词序列）：w1,w2,...,wn。

>语言模型的目标是这一整句话出现的概率是多少：$P(w1​,w2​,…,wn​)$

2）链式法则：把整局概率拆开

直接算$P(w1​,w2​,…,wn​)$很难，所以用概率里的链式法则（chain rule）拆开：
$$P\left(w_{1}, w_{2}, \ldots, w_{n}\right)=P\left(w_{1}\right) \cdot P\left(w_{2} \mid w_{1}\right) \cdot P\left(w_{3} \mid w_{1}, w_{2}\right) \cdots P\left(w_{n} \mid w_{1}, \ldots, w_{n-1}\right)$$

也就是：
>每个词出现的概率 = 在它前面所有词都已出现的条件下，它出现的概率。

3）这样做的难点是：条件太长了（历史无限长）

上面的最后一项：
$$P\left(w_{n} \mid w_{1}, \ldots, w_{n-1}\right)$$

**意思是预测当前的词要看前面所有的词**

但是现实中会出现以下问题：
- 历史长度会越来越长

- 数据里几乎不可能见过“完全一模一样的长上下文”

- 所以你无法可靠估计这么长条件概率（统计稀疏问题）

4）n-gram 的核心近似：马尔可夫假设
$$P\left(w_{t} \mid w_{1}, \ldots, w_{t-1}\right) \approx P\left(w_{t} \mid w_{t-n+1}, \ldots, w_{t-1}\right)$$

读成人话就是：

>预测第 t 个词时，我不看全部历史，只看最近的 n−1 个词。
- unigram（n=1）：只看自己，不看上下文  $P(w_t)$
- bigram（n=2）：只看前 1 个词         $P(w_t|w_{t-1})$
- trigram（n=3）：只看前 2 个词        $P(w_t|w_{t-2},w_{t-1})$

**4.n-gram的局限**

- 固定窗口、无法捕捉长距离依赖：n-gram仅考虑固定长度的局部上下文。例如，bigram模型只能看前1个词，trigram看前2个词，这限制了模型无法利用超过n-1距离的上下文。如果影响当前词/情感的关键在于更远处的词，n-gram模型将无能为力。

- 数据稀疏和特征爆炸：随着n增大，可能的n-gram组合数量呈指数增长，训练语料却不可能覆盖所有组合。结果是大多数高阶n-gram很少出现甚至未在训练中出现，模型对这些未见过的序列就难以做出可靠判断。特征空间维度也随n增大而急剧上升（被称为“维度诅咒”），增加了计算开销。

- 缺乏语义泛化：n-gram特征依然是基于词表的离散特征，对于同义词、词形变化等缺乏泛化能力。此外，每个特征被等权视为独立，不能体现词语的相对重要性或语义关系。

**5.实验
| 问题    | 比较BOW和bigram模型    |
| :-----: | :-------------------: |
| 数据    | IMDB电影评论数据集        |
| 任务    | 根据每条影评的文本预测其情感标签（positive或negative） |
| 模型    | Logistic Regression / Multinomial Naive Bayes |
| 问题类别 | 二分问题             |
| 评价指标 | 准确率（accuracy） / F1 |
| **不同点**   | 特征是否含顺序             |

In [ ]:
"""
scikit-learn：CountVectorizer + 逻辑回归 / 朴素贝叶斯
"""

# -*- coding: utf-8 -*-  # 声明源码编码，避免某些环境下中文注释乱码

# 导入正则库，用于清洗 HTML 等文本噪声
import re  # 正则表达式工具

# 导入 numpy，用于部分数组处理（可选，但常用）
import numpy as np  # 数值计算库

# 从 Hugging Face datasets 导入 load_dataset：用代码直接加载 IMDB 公共数据集
from datasets import load_dataset  # 自动下载/缓存数据集

# 导入 sklearn 的 CountVectorizer：把文本变成 unigram/bigram 计数特征
from sklearn.feature_extraction.text import CountVectorizer  # n-gram 词频特征

# 导入朴素贝叶斯分类器（适合高维稀疏词频）
from sklearn.naive_bayes import MultinomialNB  # 多项式朴素贝叶斯

# 导入逻辑回归分类器（文本分类常用线性模型）
from sklearn.linear_model import LogisticRegression  # 逻辑回归

# 导入评估指标：准确率和 F1
from sklearn.metrics import accuracy_score, f1_score  # 评价指标

# 导入 sklearn 的管道：把“向量化 + 分类器”串起来，方便对比实验
from sklearn.pipeline import Pipeline  # 机器学习流水线


def clean_text(x: str) -> str:
    # 定义文本清洗函数：输入一条文本，输出清洗后的文本
    x = x.lower()  # 全部转小写（与你描述的预处理一致）
    x = re.sub(r"<br\s*/?>", " ", x)  # 把常见的 IMDB 换行标签 <br/> 替换为空格
    x = re.sub(r"<.*?>", " ", x)  # 粗略去除其它 HTML 标签（非严格，但足够实验）
    x = re.sub(r"[^a-z0-9\s']", " ", x)  # 移除标点符号（保留字母数字空格和撇号，利于 not / don't）
    x = re.sub(r"\s+", " ", x).strip()  # 多个空白合并成一个空格，并去掉首尾空格
    return x  # 返回清洗后的文本


def run_sklearn_experiment(
    ngram_range=(1, 1),  # 指定 unigram 或 bigram
    max_features=20000,  # 限制特征维度，避免 bigram 爆炸（你文档里也提到要限制）
    model_name="logreg",  # 选择分类器：logreg 或 nb
):
    # 加载 IMDB 数据集（首次运行会自动下载到本地缓存；不需要你手动下载）
    ds = load_dataset("imdb")  # 返回一个 DatasetDict，含 train/test

    # 取出训练文本列表
    train_texts = [clean_text(t) for t in ds["train"]["text"]]  # 对训练集逐条清洗
    # 取出训练标签列表（0=neg, 1=pos）
    train_labels = np.array(ds["train"]["label"], dtype=np.int64)  # 转成 numpy 数组便于 sklearn 使用

    # 取出测试文本列表
    test_texts = [clean_text(t) for t in ds["test"]["text"]]  # 对测试集逐条清洗
    # 取出测试标签列表
    test_labels = np.array(ds["test"]["label"], dtype=np.int64)  # 转成 numpy 数组

    # 创建 CountVectorizer：把文本变成“n-gram 计数”稀疏矩阵
    vectorizer = CountVectorizer(
        ngram_range=ngram_range,  # (1,1) = unigram；(2,2) = bigram
        max_features=max_features,  # 只保留频率最高的前 max_features 个特征
        lowercase=False,  # 我们已经手动 lower() 了，这里不重复做
        stop_words=None,  # 不移除停用词（保留 not/no 等组合信息）
    )

    # 根据 model_name 选择分类器
    if model_name == "nb":  # 如果选择朴素贝叶斯
        clf = MultinomialNB()  # 朴素贝叶斯通常对词频计数效果不错
    else:  # 否则默认逻辑回归
        clf = LogisticRegression(
            max_iter=1000,  # 增大迭代次数，保证收敛（文本维度高常需要更大 max_iter）
            n_jobs=None,  # 不强制多线程（不同环境兼容性更好）
        )

    # 用 Pipeline 串起来：先向量化，再分类
    pipe = Pipeline([  # 创建一个流水线对象
        ("vec", vectorizer),  # 第一步：CountVectorizer
        ("clf", clf),  # 第二步：分类器
    ])

    # 在训练集上训练（内部会先 fit 向量器，再 fit 分类器）
    pipe.fit(train_texts, train_labels)  # 训练模型

    # 在测试集上预测标签
    pred = pipe.predict(test_texts)  # 输出预测类别（0/1）

    # 计算准确率
    acc = accuracy_score(test_labels, pred)  # accuracy
    # 计算 F1（二分类用 binary，pos_label=1 对应 positive）
    f1 = f1_score(test_labels, pred, average="binary", pos_label=1)  # F1

    # 打印实验配置与结果
    print("=" * 80)  # 分割线
    print(f"ngram_range={ngram_range}, max_features={max_features}, model={model_name}")  # 输出配置
    print(f"Accuracy = {acc:.4f}")  # 输出准确率
    print(f"F1       = {f1:.4f}")  # 输出 F1


if __name__ == "__main__":
    # 跑 unigram + 逻辑回归（你的方案1）
    run_sklearn_experiment(ngram_range=(1, 1), max_features=20000, model_name="logreg")  # unigram/logreg

    # 跑 bigram + 逻辑回归（你的方案2）
    run_sklearn_experiment(ngram_range=(2, 2), max_features=20000, model_name="logreg")  # bigram/logreg

    # （可选）同样再对比朴素贝叶斯
    run_sklearn_experiment(ngram_range=(1, 1), max_features=20000, model_name="nb")  # unigram/nb
    run_sklearn_experiment(ngram_range=(2, 2), max_features=20000, model_name="nb")  # bigram/nb


d:\Anaconda_envs\envs\pytorch2.3.1\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\weixi\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 346695.72 examples/s]


ngram_range=(1, 1), max_features=20000, model=logreg
Accuracy = 0.8622
F1       = 0.8609
ngram_range=(2, 2), max_features=20000, model=logreg
Accuracy = 0.8595
F1       = 0.8606
ngram_range=(1, 1), max_features=20000, model=nb
Accuracy = 0.8170
F1       = 0.8055
ngram_range=(2, 2), max_features=20000, model=nb
Accuracy = 0.8615
F1       = 0.8615


In [ ]:
"""
使用pytorch完成
"""

# -*- coding: utf-8 -*-  # 声明源码编码

# 导入正则库：清洗文本
import re  # 正则表达式

# 导入 numpy：中间转换用
import numpy as np  # 数组与数值处理

# 导入 PyTorch：训练逻辑回归
import torch  # PyTorch 主库
import torch.nn as nn  # 神经网络模块
import torch.optim as optim  # 优化器

# 导入 Hugging Face datasets：直接引用 IMDB 公共数据集（自动下载/缓存）
from datasets import load_dataset  # 数据集加载

# 导入 CountVectorizer：提取 unigram/bigram 计数特征
from sklearn.feature_extraction.text import CountVectorizer  # 词频向量化

# 导入评估指标：accuracy、f1
from sklearn.metrics import accuracy_score, f1_score  # 指标计算


def clean_text(x: str) -> str:
    # 文本清洗函数（与 sklearn 版本保持一致，公平对比）
    x = x.lower()  # 小写化
    x = re.sub(r"<br\s*/?>", " ", x)  # 替换 <br/> 换行标签
    x = re.sub(r"<.*?>", " ", x)  # 去除其它 HTML 标签
    x = re.sub(r"[^a-z0-9\s']", " ", x)  # 去标点（保留撇号）
    x = re.sub(r"\s+", " ", x).strip()  # 合并多余空白
    return x  # 返回清洗后的文本


def scipy_csr_to_torch_sparse(csr):
    # 把 scipy 的 CSR 稀疏矩阵转换成 PyTorch 稀疏 COO 张量
    coo = csr.tocoo()  # 转成 COO 格式（行、列、值显式列出）
    i = torch.tensor(np.vstack([coo.row, coo.col]), dtype=torch.long)  # 2 x nnz 的 index 张量
    v = torch.tensor(coo.data, dtype=torch.float32)  # nnz 的 value 张量（用 float 便于训练）
    shape = torch.Size(coo.shape)  # 稀疏张量形状
    return torch.sparse_coo_tensor(i, v, shape).coalesce()  # 返回 PyTorch 稀疏张量并合并重复索引


class SparseLogReg(nn.Module):
    # 定义“稀疏输入的逻辑回归”模型：y = sigmoid(Wx + b)
    def __init__(self, num_features):
        super().__init__()  # 初始化父类
        self.weight = nn.Parameter(torch.zeros(num_features, 1))  # 权重向量 W（F x 1）
        self.bias = nn.Parameter(torch.zeros(1))  # 偏置 b（标量）

    def forward(self, x_sparse):
        # 前向传播：输入是稀疏张量 x_sparse，形状 (B, F)
        logits = torch.sparse.mm(x_sparse, self.weight).squeeze(1) + self.bias  # 稀疏矩阵乘法得到 (B,)
        return logits  # 返回未过 sigmoid 的 logits（更适配 BCEWithLogitsLoss）


def run_torch_experiment(
    ngram_range=(1, 1),  # (1,1)=unigram；(2,2)=bigram
    max_features=20000,  # 限制特征数
    batch_size=256,  # 批大小（稀疏输入通常选中等 batch）
    epochs=3,  # 训练轮数（IMDB 较大，逻辑回归几轮就够看到效果）
    lr=0.1,  # 学习率
    device="cpu",  # 设备：cpu 或 cuda
):
    # 加载 IMDB 数据集（自动下载/缓存）
    ds = load_dataset("imdb")  # 获取 train/test

    # 清洗训练文本
    train_texts = [clean_text(t) for t in ds["train"]["text"]]  # 逐条清洗
    # 训练标签（0/1）
    y_train = np.array(ds["train"]["label"], dtype=np.float32)  # BCE loss 常用 float

    # 清洗测试文本
    test_texts = [clean_text(t) for t in ds["test"]["text"]]  # 逐条清洗
    # 测试标签
    y_test = np.array(ds["test"]["label"], dtype=np.float32)  # 转 float

    # 构造向量器（n-gram 计数特征）
    vectorizer = CountVectorizer(
        ngram_range=ngram_range,  # 指定 unigram/bigram
        max_features=max_features,  # 限制维度
        lowercase=False,  # 已经手动 lower
        stop_words=None,  # 不移除停用词
    )

    # 在训练集上拟合并转换：得到 X_train 稀疏矩阵 (N_train, F)
    X_train = vectorizer.fit_transform(train_texts)  # CSR 稀疏矩阵
    # 在测试集上只转换：得到 X_test (N_test, F)
    X_test = vectorizer.transform(test_texts)  # CSR 稀疏矩阵

    # 获取特征维度 F
    num_features = X_train.shape[1]  # 向量维度

    # 把 CSR 转成 PyTorch 稀疏张量
    X_train_t = scipy_csr_to_torch_sparse(X_train).to(device)  # 放到 device
    # 把测试集也转成 PyTorch 稀疏张量
    X_test_t = scipy_csr_to_torch_sparse(X_test).to(device)  # 放到 device

    # 把标签转成 torch 张量
    y_train_t = torch.tensor(y_train, dtype=torch.float32, device=device)  # (N_train,)
    # 测试标签张量
    y_test_t = torch.tensor(y_test, dtype=torch.float32, device=device)  # (N_test,)

    # 初始化模型
    model = SparseLogReg(num_features=num_features).to(device)  # 创建并移动到 device

    # 定义损失：二分类用 BCEWithLogitsLoss（内部带 sigmoid，更稳定）
    criterion = nn.BCEWithLogitsLoss()  # 二分类交叉熵（logits 版）

    # 定义优化器：SGD 对逻辑回归足够，也可换 Adam
    optimizer = optim.SGD(model.parameters(), lr=lr)  # 随机梯度下降

    # 获取训练样本数
    n_train = X_train_t.size(0)  # 训练样本数量

    # 开始训练若干轮
    for epoch in range(epochs):  # 遍历 epoch
        model.train()  # 切到训练模式
        total_loss = 0.0  # 累计 loss
        num_batches = 0  # 统计 batch 数

        # 做一个随机打乱的索引（逻辑回归也可以打乱训练）
        perm = torch.randperm(n_train, device=device)  # 随机排列索引

        # 按 batch_size 切分索引
        for start in range(0, n_train, batch_size):  # 从 0 到 n_train 步进 batch_size
            end = min(start + batch_size, n_train)  # 计算 batch 结尾（不越界）
            idx = perm[start:end]  # 取当前 batch 的索引

            # 从稀疏矩阵中取子集：PyTorch 稀疏张量不支持花式切片得很完美，
            # 这里用 index_select 在维度0取行（可用）
            xb = torch.index_select(X_train_t, 0, idx)  # 得到 (B, F) 的稀疏张量
            yb = y_train_t[idx]  # 得到 (B,) 标签

            optimizer.zero_grad()  # 清空梯度
            logits = model(xb)  # 前向：得到 (B,) logits
            loss = criterion(logits, yb)  # 计算 loss
            loss.backward()  # 反向传播
            optimizer.step()  # 参数更新

            total_loss += float(loss.item())  # 累加 batch loss
            num_batches += 1  # batch 数加一

        # 每个 epoch 打印平均 loss
        print(f"[epoch {epoch+1}/{epochs}] avg_loss = {total_loss / max(1, num_batches):.4f}")  # 输出训练损失

    # 开始评估
    model.eval()  # 切到评估模式
    with torch.no_grad():  # 评估不计算梯度
        test_logits = model(X_test_t)  # 得到测试 logits（N_test,)
        test_prob = torch.sigmoid(test_logits)  # 转成概率（0~1）
        test_pred = (test_prob >= 0.5).to(torch.int64).cpu().numpy()  # 阈值 0.5 转成预测类别

    # 取出真实标签（numpy）
    y_true = y_test.astype(np.int64)  # 转 int 便于 sklearn 指标
    # 计算 accuracy
    acc = accuracy_score(y_true, test_pred)  # 准确率
    # 计算 F1
    f1 = f1_score(y_true, test_pred, average="binary", pos_label=1)  # F1（positive=1）

    # 打印结果
    print("=" * 80)  # 分割线
    print(f"[PyTorch LogReg] ngram_range={ngram_range}, max_features={max_features}")  # 输出配置
    print(f"Accuracy = {acc:.4f}")  # 输出准确率
    print(f"F1       = {f1:.4f}")  # 输出 F1


if __name__ == "__main__":
    # 自动选择设备：有 GPU 就用 GPU，没有就用 CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"  # 设备选择

    # 跑 unigram 实验（方案1）
    run_torch_experiment(ngram_range=(1, 1), max_features=20000, epochs=3, lr=0.1, device=device)  # unigram

    # 跑 bigram 实验（方案2）
    run_torch_experiment(ngram_range=(2, 2), max_features=20000, epochs=3, lr=0.1, device=device)  # bigram


d:\Anaconda_envs\envs\pytorch2.3.1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/imdb/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x1e5e3b8e130>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: ac12d4e6-3d6b-49d2-b4b0-9a0a22923f4b)')' thrown while requesting HEAD https://huggingface.co/datasets/imdb/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/imdb/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x1e5e3b8e2e0>, 'Connection to hugging

[epoch 1/3] avg_loss = 3.5759
[epoch 2/3] avg_loss = 2.6045
[epoch 3/3] avg_loss = 2.0926
[PyTorch LogReg] ngram_range=(1, 1), max_features=20000
Accuracy = 0.7596
F1       = 0.7130


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/imdb/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x1e5e4aa5580>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 5dd94788-71e4-4e6c-b74e-6f1678c1ee03)')' thrown while requesting HEAD https://huggingface.co/datasets/imdb/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/imdb/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x1e5e4aa56d0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 509d63ee-700f-45aa-89c2-81b5172cb0e2)')' thrown while requesting HEAD https://huggingface.co/datasets/imdb/resolve/main/README.md
Retrying in 2s [Retry 2/5].
'(MaxRetryError("HTTPSConnectionPool(hos

[epoch 1/3] avg_loss = 0.6126
[epoch 2/3] avg_loss = 0.5272
[epoch 3/3] avg_loss = 0.4831
[PyTorch LogReg] ngram_range=(2, 2), max_features=20000
Accuracy = 0.8141
F1       = 0.8150
